## May be of interest
### http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
### http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
### http://scikit-learn.org/stable/modules/cross_validation.html

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
np.random.seed(0)

In [2]:
#Loading data
df_train = pd.read_csv('../input/train_users_2.csv')
df_test = pd.read_csv('../input/test_users.csv')
labels = df_train['country_destination'].values
df_train = df_train.drop(['country_destination'], axis=1)
id_test = df_test['id']
piv_train = df_train.shape[0]

In [3]:
#Creating a DataFrame with train+test data
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
#Removing id and date_first_booking
df_all = df_all.drop(['id', 'date_first_booking'], axis=1)
#Filling nan
df_all = df_all.fillna(-1)

In [4]:
#####Feature engineering#######
#date_account_created
dac = np.vstack(df_all.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
df_all['dac_year'] = dac[:,0]
df_all['dac_date'] = 100*dac[:,1]+dac[:,2]
df_all = df_all.drop(['date_account_created'], axis=1)
df_all.head()

,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_date
0,20090319043255,-unknown-,-1,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,628
1,20090523174809,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2011,525
2,20090609231247,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2010,928
3,20091031060129,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2011,1205
4,20091208061105,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,914


In [5]:
#timestamp_first_active
tfa = np.vstack(df_all.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
df_all['tfa_year'] = tfa[:,0]
df_all['tfa_date'] = 100*tfa[:,1]+tfa[:,2]
df_all = df_all.drop(['timestamp_first_active'], axis=1)

In [6]:
#Age
av = df_all.age.values
df_all['age'] = np.where(np.logical_or(av<14, av>100), -1, av)

In [7]:
#One-hot-encoding features
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    df_all_dummy = pd.get_dummies(df_all[f], prefix=f)
    df_all = df_all.drop([f], axis=1)
    df_all = pd.concat((df_all, df_all_dummy), axis=1)

In [8]:
#Splitting train and test
vals = df_all.values
X = vals[:piv_train]
le = LabelEncoder()
y = le.fit_transform(labels)   
X_test = vals[piv_train:]

In [11]:
#Classifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(X, y)
y_pred = xgb.predict_proba(X_test)  
y_pred

array([[ 0.00265829,  0.00387519,  0.00329876, ...,  0.00189142,
         0.23067766,  0.0316067 ],
       [ 0.00116007,  0.00176287,  0.00183053, ...,  0.00102232,
         0.07198177,  0.01095488],
       [ 0.00194602,  0.00565485,  0.00361986, ...,  0.00160796,
         0.13355483,  0.03082748],
       ..., 
       [ 0.00317015,  0.00329895,  0.00411754, ...,  0.0018761 ,
         0.11694062,  0.02139158],
       [ 0.00290426,  0.00397507,  0.00442984, ...,  0.00138281,
         0.12365146,  0.02294582],
       [ 0.00549911,  0.00775737,  0.00829915, ...,  0.00189665,
         0.41571447,  0.05309358]], dtype=float32)

In [12]:
#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx]*5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

In [13]:
#Generate submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.head(50)

,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,other
3,5uwns89zht,FR
4,5uwns89zht,IT
5,jtl0dijy2j,NDF
6,jtl0dijy2j,US
7,jtl0dijy2j,other
8,jtl0dijy2j,FR
9,jtl0dijy2j,IT


In [ ]:
sub.to_csv('../output/sub.csv',index=False)